I have a prototype of they system for corrupting sentences, for different semantic meaning:

My process is:

1. Tokenize the sentence (Currently using the NLTK regex tokenizer, it seem sufficient)
2. Parts of Speech tag (Currently using the Stanford POS Tagger (via NLTK))
3. For each word except auxiliary verbs, and words that make up phrases (phrases are checked for by querying wordnet)
        1. Use WordNet to find antonyms of the same POS tag  (So "race" (Noun, as in a competition) has no antonyms, but "race" (Verb, to race the train) has "linger" as an antonym.
        2. Unstem: WordNet stemming/lemmaisation (of the antonym) removes Tense, Plurality, comparativeness, and superlativeness, so I make use of the POS tag of the original to work those out, then restore them using the Pattern library's tools for this (http://jmlr.csail.mit.edu/papers/volume13/desmedt12a/desmedt12a.pdf)
        3. Remove any generated words that are not real (Failures of the Unstemming process, eg saless), by checking against the SCOWL british-insane word list.
        4. I remove any suggested antonyms that are short phrases (eg Wordnet suggests that "take_away" is an antonym of "add", however as adding a work word change the structure of the sentence.)
        5. select one randomly if there are multiple.
4. I incrementally subsitute one addional antynym until I have run out of words with antonys, saving the sentence at each step. (What I'm currentl;y callign each sentence of different corruption level)
5. I repair the indefinite articles ('an' vs 'a')

I can check the final sentence by sending it through the POS tagger and seeing if I get the same tags.
This is not perfect. Its not bad though.
It got a lot better when I changed to using the Stanford POS tagger, as it was more able to tag and retag correctly and thus was most consistent.

The whole method is very heurustic.
It's not too bad but makes a lot of unnatural sounding sentences.
And will sometimes choose the wrong antonym.

For example stating with the sentence:
 - It's not too bad but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural sounding sentences.
 - It 's not too unregretful but makes a lot of unnatural devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing sentences.
 - It 's not too unregretful but makes a lot of unaffected devoicing acquittals.

In [1]:
from SemanticCorruption import *

In [2]:
from nltk.corpus import wordnet as wn
import pattern.en as en

In [3]:
sents = ["It's not too bad but makes a lot of unnatural sounding sentences.",
'A Washington County man may have the countys first human case of West Nile virus, the health department said Friday',
"It was a dusty, dry, hot day, and the flys were buzzing.",
"The article is the most common determiner (DT) in English.",
"We may have a question",
"Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?",
"Both gerunds and infinitives can be used as the subject or the complement of a sentence.",
"Shares of Xoma fell 16 percent in early trade, while shares of Genentech, a much larger company with several products on the market, were up 2 percent.",
"Six months ago, the IMF and Argentina struck a bare-minimum $6.8-billion debt rollover deal that expires in August.",
"He plans to have dinner with troops at Kosovo's U.S. military headquarters, Camp Bondsteel.",
"After that, he plans to have dinner at Camp Bondsteel with U.S. troops stationed there.",
"He added that prosecutors will seek the death penalty.",
"Who is this man?",
"Who is that man?",
"This evil thief stole that car!",
"The motorist is angry, so the pedestrian is understandably scared",
"The hero was brave, he defeated the dragon.",
"The villian was cowardly, he was slain by the dragon.",
"War and Peace opens in the Russian city of St. Petersburg in 1805, as Napoleon's conquest of western Europe is just beginning to stir fears in Russia.",
"PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer."
        ]

In [4]:
def leveled_semantic_corrupt_verb_antonym_sentences(sent):
    return leveled_semantic_corrupt_sentences(sent, get_verb_antos)

def leveled_semantic_corrupt_noun_synonym_sentences(sent):
    return leveled_semantic_corrupt_sentences(sent, get_noun_synonyms)

In [5]:
sent = "PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will report directly to Mr So."

In [6]:

print(sent)
print("\n".join(list(leveled_semantic_corrupt_sentences(sent, get_noun_synonyms))))
print("-"*50)

PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will report directly to Mr So.
PCCW 's chief operating officeholder , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .
PCCW 's gaffer operating officeholder , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .
PCCW 's gaffer operating officeholder , Mike Butcher , and Alex Arena , the chief financial policeman , will report directly to Mr So .
PCCW 's gaffer operating officeholder , Mike Butcher , and Alex Arena , the chief financial policeman , will report directly to Mr. So .
--------------------------------------------------


In [29]:
leveled_semantic_corrupt_sentences(sent, get_noun_synonyms)

In [28]:
valid_words = set(open("/usr/share/dict/british-english-insane").read().split())

In [34]:
"Alexis" in setofwords

True

In [19]:
s=wn.synsets("finsss")[0]


In [20]:
s.definition()

u'the cardinal number that is the sum of four and one'

In [10]:
for sent in sents:
    tagged_phrases = get_tagged_phrases(tokenize_and_tag(sent)[1],3)
    phrases = zip(*tagged_phrases)[0]
    print " ".join(phrases)

It 's not too_bad but makes a_lot of unnatural sounding sentences .
A Washington County man may have the countys first human case of West_Nile_virus , the health department said Friday
It was a dusty , dry , hot day , and the flys were buzzing .
The article is the most common determiner ( DT ) in English .
We may have a question
Is changing an odd number of verbs , adverbs , adjectives and nouns to their antonyms expected to produce a semantically distant sentence ?
Both gerunds and infinitives can be used as the subject or the complement of a sentence .
Shares of Xoma fell 16 percent in early trade , while shares of Genentech , a much larger company with several products on the market , were up 2 percent .
Six months ago , the IMF and Argentina struck a bare-minimum $ 6.8-billion debt rollover deal that expires in August .
He plans to have dinner with troops at Kosovo 's U.S. military_headquarters , Camp Bondsteel .
After that , he plans to have dinner at Camp Bondsteel with U.S. troo

In [24]:
for sent in sents:
    indexes = list(get_phrases_indexes(tokenize_and_tag(sent)[1],3))
    indexes.sort()
    print indexes

[3, 4, 7, 8]
[12, 13, 14]
[]
[]
[]
[]
[]
[]
[]
[11, 12]
[]
[7, 8]
[]
[]
[]
[]
[]
[]
[9, 10]
[2, 3, 4, 15, 16, 17, 24, 25]


In [15]:
a= set()

In [20]:
a.update(["21"])

In [21]:
a

{'21'}